# PPR 

<img src="{{site.baseurl}}/images/ppr4r.png" width = "100%">

# List being Created:

    def get(self):
    chats = quizgrading.query.all()
    allChats = []  
    for i in range(len(chats)):
        allChats.append(chats[i].read())  

    return jsonify(allChats)

**Explanation:** The get method retrieves all records from the quizgrading table using quizgrading.query.all(). It then iterates over the records, calling the .read() method on each to convert them into dictionaries. Then, it returns these dictionaries as a JSON response using jsonify(allChats), which is sent back to the client.

# List being processed:

    for i in range(len(chats)):
        allChats.append(chats[i].read())
    
**Explanation:** This loop iterates over chats, calls the read() method on each quizgrading object, and appends the resulting dictionary to allChats.

# A Function:
    async function loadAttempts() {
    const currentUserResponse = await fetch(${pythonURI}/api/id, fetchOptions);
    if (!currentUserResponse.ok) throw new Error('Failed to fetch current user');
    const currentUser = await currentUserResponse.json();
    userName = currentUser.uid;
    userID = currentUser.id;

    const quizGrading = await fetch(quizGradingsApi, fetchOptions);
    if (!quizGrading.ok) { console.error("Error loading attempts:", quizGrading); }

    const quizResults = await quizGrading.json();
    console.log(quizResults);

    const table = document.getElementById('attemptsTable');
    let thead = table.querySelector('thead');
    let tbody = table.querySelector('tbody');

    if (!thead) {
        thead = document.createElement('thead');
        const headerRow = document.createElement('tr');
        const headers = ['ID', 'Username', 'Quiz Grade', 'Attempt Date', 'Actions'];

        headers.forEach(headerText => {
            const th = document.createElement('th');
            th.textContent = headerText;
            headerRow.appendChild(th);
        });

        thead.appendChild(headerRow);
        table.appendChild(thead);
    }

    if (!tbody) {
        tbody = document.createElement('tbody');
        table.appendChild(tbody);
    }

    tbody.innerHTML = '';
   
    quizResults.forEach(attempt => {
        const row = document.createElement('tr');
        const idCell = document.createElement('td');
        idCell.innerHTML = attempt.id;
        const usernameCell = document.createElement('td');
        usernameCell.innerHTML = attempt.username;
        const quizgradeCell = document.createElement('td');
        quizgradeCell.innerHTML = attempt.quizgrade;
        const attemptCell = document.createElement('td');
        attemptCell.innerHTML = attempt.attempt;
        const actionCell = document.createElement('td'); 

        if (attempt.username === userName) {
            const deleteButton = document.createElement('button');
            deleteButton.innerHTML = 'Delete';
            deleteButton.addEventListener('click', () => deleteAttempt(attempt.id));
            const editButton = document.createElement('button');
            editButton.innerHTML = 'Edit';
            editButton.addEventListener('click', () => editAttempt(attempt.id));
            actionCell.append(deleteButton);
            actionCell.append(editButton);
        }

        row.append(idCell);
        row.append(usernameCell);
        row.append(quizgradeCell);
        row.append(attemptCell);
        row.append(actionCell);
        tbody.append(row);
    });
}

**Explanation:** 
Sequencing: The steps are executed in order, for example, fetching the quiz grading data and updating the table.
Selection: The if statements (for example, checking if the thead or tbody elements exist) represent selection.
Iteration: The forEach() loop iterates through each quiz result to create rows in the table.

# Call to Function:

    window.onload = () => {
    const selectedQuestions = randomizeQuestions(Questions, 5);
    buildQuiz(selectedQuestions);
    loadAttempts();
    };


**Explanation:** The loadAttempts function is called and ensures that the quiz attempts and scores are displayed in the table after each attempt and reload of the screen. This function is invoked when the window is loaded (window.onload), triggering the fetching of the attempts and rendering them in the table.